In [21]:
from src.utils import audio_to_mel_spectrogram
from src.models.mel_spec_vqvae import MelSpecVQVAE

from src.wrappers.null_dafx_wrapper import NullDAFXWrapper
from src.plot_utils import get_audio_dataset

In [22]:
CHECKPOINT = "/home/kieran/Level5ProjectAudioVAE/src/l5proj_MelSpecVQVAE/ck905h93/checkpoints/epoch=37-step=11894.ckpt"

In [24]:
model = MelSpecVQVAE.load_from_checkpoint(CHECKPOINT)
model.eval()

MelSpecVQVAE(
  (_encoder): Encoder(
    (_conv_1): Conv2d(1, 32, kernel_size=(4, 4), stride=(3, 3), padding=(1, 1))
    (_conv_2): Conv2d(32, 64, kernel_size=(4, 4), stride=(3, 3), padding=(1, 1))
    (_conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (_residual_stack): ResidualStack(
      (_layers): ModuleList(
        (0): Residual(
          (_block): Sequential(
            (0): ReLU(inplace=True)
            (1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): ReLU(inplace=True)
            (3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
        )
        (1): Residual(
          (_block): Sequential(
            (0): ReLU(inplace=True)
            (1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): ReLU(inplace=True)
            (3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
        )
        

In [27]:
dataset = get_audio_dataset(NullDAFXWrapper(), num_examples_per_epoch=1, length=model.hparams.train_length)

x = next(iter(dataset))
X = audio_to_mel_spectrogram(signal=x,
                             sample_rate=model.hparams.sample_rate,
                             n_mels=model.hparams.n_mels,
                             n_fft=model.hparams.n_fft,
                             win_length=model.hparams.win_length,
                             f_max=model.hparams.f_max,
                             f_min=model.hparams.f_min)

z = model._encoder(X)
z = model._pre_vq_conv(z)
loss, quantized, perplexity, enc = model._vq_vae(z)

100%|████████████████████████████████████████| 88/88 [00:00<00:00, 43341.80it/s]


Loaded 88 files for train = 66.89 hours.


In [28]:
quantized.shape

torch.Size([1, 2, 28, 28])